In [ ]:
import cv2
import numpy as np
import os
from pathlib import Path
import mahotas as mt
from sklearn.svm import LinearSVC
from typing import List, Tuple

In [ ]:
def extract_features(image: np.ndarray) -> np.ndarray:
    textures = mt.features.haralick(image)
    ht_mean = textures.mean(axis=0)
    return ht_mean

def load_training_data(train_path: str) -> Tuple[np.ndarray, np.ndarray]:
    train_features = []
    train_labels = []

    # Menggunakan Path untuk handling path yang lebih baik
    train_dir = Path(train_path)

    print("Starting feature extraction...")
    for class_dir in train_dir.iterdir():
        if class_dir.is_dir():
            cur_label = class_dir.name

            for i, img_path in enumerate(class_dir.glob("*.jpg"), 1):
                print(f"Processing Image - {i} in {cur_label}")

                # Baca gambar
                image = cv2.imread(str(img_path))
                if image is None:
                    print(f"Warning: Could not read image {img_path}")
                    continue

                # Konversi ke grayscale
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                # Ekstrak fitur
                features = extract_features(gray)

                train_features.append(features)
                train_labels.append(cur_label)

    return np.array(train_features), np.array(train_labels)

def predict_image(model: LinearSVC, image_path: str) -> Tuple[str, np.ndarray]:
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not read image: {image_path}")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = extract_features(gray)

    prediction = model.predict(features.reshape(1, -1))[0]

    # Tambahkan prediksi ke gambar
    cv2.putText(
        image,
        prediction,
        (20, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.0,
        (0, 255, 255),
        3
    )

    return prediction, image

In [ ]:
def main():
    # Path setup
    train_path = "dataset/train"
    test_path = "dataset/test"

    # Load training data
    try:
        train_features, train_labels = load_training_data(train_path)
        print(f"Training features shape: {train_features.shape}")
        print(f"Training labels shape: {train_labels.shape}")
    except Exception as e:
        print(f"Error loading training data: {e}")
        return

    # Train classifier
    print("Creating and training classifier...")
    clf_svm = LinearSVC(random_state=9, max_iter=2000)  # Increased max_iter
    clf_svm.fit(train_features, train_labels)

    # Process test images
    test_dir = Path(test_path)
    for img_path in test_dir.glob("*.jpg"):
        try:
            prediction, processed_image = predict_image(clf_svm, str(img_path))
            print(f"Prediction for {img_path.name}: {prediction}")

            # Display result
            cv2.imshow("Test Image", processed_image)
            key = cv2.waitKey(0) & 0xFF
            if key == 27:  # ESC key
                break

        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            continue

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()